In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('baitap.db')
cursor = conn.cursor()

## Câu 1

In [3]:
# Tạo bảng ví dụ câu 1
cursor.execute('''CREATE TABLE IF NOT EXISTS Cau1(
               ID INT,
               A FLOAT,
               B FLOAT
               );

''')

In [4]:
# Thêm dữ liệu ví dụ
data = [
    (1, 20, 35.5),
    (2, 25, 33.3),
    (3, 30, 40.1),
    (4, 45.2, 50),
    (5, 55.5, 64.9)
]

cursor.executemany("INSERT INTO Cau1 (ID, A, B) VALUES (?, ?, ?)", data)
conn.commit()

In [5]:
for row in cursor.execute("SELECT * FROM Cau1"):
    print(row)

(1, 20.0, 35.5)
(2, 25.0, 33.3)
(3, 30.0, 40.1)
(4, 45.2, 50.0)
(5, 55.5, 64.9)


In [6]:
cursor.execute('''
    SELECT
        COUNT(*) as n,
        SUM(A) as sum_A,
        SUM(B) as sum_B,
        SUM(A * B) as sum_AB,
        SUM(A * A) as sum_A2,
        SUM(B * B) as sum_B2
    FROM Cau1
''')

n, sum_A, sum_B, sum_AB, sum_A2, sum_B2 = cursor.fetchone()

numerator = n * sum_AB - sum_A * sum_B
denominator = ((n * sum_A2 - sum_A ** 2) ** 0.5) * ((n * sum_B2 - sum_B ** 2) ** 0.5)

r = numerator / denominator if denominator != 0 else None
print(f"Hệ số tương quan giữa A và B: R(AB)={r}")

Hệ số tương quan giữa A và B: R(AB)=0.9696412793790654


> Hệ số tương quan cao, tuyến tính dương

## Câu 2

In [7]:
# Tạo bảng câu 2(bảng gốc)
cursor.execute('''CREATE TABLE IF NOT EXISTS Banggoc (
                Day Text,
                A FLOAT,
                B FLOAT,
                C FLOAT
                );

''')

# Thêm dữ liệu gốc
data = [
    ('Day 1', 8, 9, 7),
    ('Day 2', 7.5, 8.5, 7),
    ('Day 3', 6, 7, 8),
    ('Day 4', 7, 6, 5)
]

cursor.executemany("INSERT INTO Banggoc (Day, A, B, C) VALUES (?, ?, ?, ?)", data)
conn.commit()

In [8]:
for row in cursor.execute("SELECT * FROM Banggoc"):
    print(row)

('Day 1', 8.0, 9.0, 7.0)
('Day 2', 7.5, 8.5, 7.0)
('Day 3', 6.0, 7.0, 8.0)
('Day 4', 7.0, 6.0, 5.0)


In [9]:
# Tạo bảng câu 2(bảng dạng quan hệ)
cursor.execute('''CREATE TABLE IF NOT EXISTS CarScores (
                Day INT,
                Car TEXT,
                Score FLOAT
                );

''')

# Thêm dữ liệu thay đổi
data = [
    (1, 'A', 8),    
    (1, 'B', 9),    
    (1, 'C', 7),
    (2, 'A', 7.5),  
    (2, 'B', 8.5),  
    (2, 'C', 7),
    (3, 'A', 6),    
    (3, 'B', 7),    
    (3, 'C', 8),
    (4, 'A', 7),    
    (4, 'B', 6),    
    (4, 'C', 5)
]

# Chèn dữ liệu vào bảng
cursor.executemany("INSERT INTO CarScores (Day, Car, Score) VALUES (?, ?, ?)", data)
conn.commit()

In [10]:
for row in cursor.execute("SELECT * FROM CarScores"):
    print(row)

(1, 'A', 8.0)
(1, 'B', 9.0)
(1, 'C', 7.0)
(2, 'A', 7.5)
(2, 'B', 8.5)
(2, 'C', 7.0)
(3, 'A', 6.0)
(3, 'B', 7.0)
(3, 'C', 8.0)
(4, 'A', 7.0)
(4, 'B', 6.0)
(4, 'C', 5.0)


In [11]:
cursor.execute('''
    SELECT 
        Car AS TenXe,
        CASE 
            WHEN Score > 8 THEN 'Cao'
            WHEN Score > 6 THEN 'TrungBinh'
            ELSE 'Thap'
        END AS MucDiem,
        COUNT(*) AS TongSo
    FROM CarScores
    GROUP BY TenXe, MucDiem
    ORDER BY TenXe, MucDiem;
''')

for row in cursor.fetchall():
    print(f"Mẫu xe {row[0]} có {row[2]} lần được đánh giá '{row[1]}'")

Mẫu xe A có 1 lần được đánh giá 'Thap'
Mẫu xe A có 3 lần được đánh giá 'TrungBinh'
Mẫu xe B có 2 lần được đánh giá 'Cao'
Mẫu xe B có 1 lần được đánh giá 'Thap'
Mẫu xe B có 1 lần được đánh giá 'TrungBinh'
Mẫu xe C có 1 lần được đánh giá 'Thap'
Mẫu xe C có 3 lần được đánh giá 'TrungBinh'


In [12]:
import pandas as pd
from scipy.stats import f_oneway

# Đọc dữ liệu từ SQLite
df = pd.read_sql_query("SELECT * FROM CarScores", conn)
df

,Day,Car,Score
0,1,A,8.0
1,1,B,9.0
2,1,C,7.0
3,2,A,7.5
4,2,B,8.5
5,2,C,7.0
6,3,A,6.0
7,3,B,7.0
8,3,C,8.0
9,4,A,7.0


In [13]:
# Dùng bảng ANOVA
# Gom điểm theo từng mẫu xe
scores_A = df[df['Car'] == 'A']['Score']
scores_B = df[df['Car'] == 'B']['Score']
scores_C = df[df['Car'] == 'C']['Score']

# Kiểm định ANOVA
f_stat, p_value = f_oneway(scores_A, scores_B, scores_C)

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < 0.05:
    print("Có sự khác biệt đáng kể giữa các mẫu xe.")
else:
    print("Không có sự khác biệt đáng kể giữa các mẫu xe.")

F-statistic: 0.5495
P-value: 0.5954
Không có sự khác biệt đáng kể giữa các mẫu xe.


In [14]:
# Dùng kiểm định Chi-Square(χ2).
from scipy.stats import chi2_contingency
# Thêm cột 'PhanLoai' theo mức điểm
df['PhanLoai'] = pd.cut(df['Score'], 
                        bins=[-float('inf'), 6, 8, float('inf')],
                        labels=['Thap', 'TrungBinh', 'Cao'])

# Bảng chéo 1: Phân loại vs Xe
table_car = pd.crosstab(df['PhanLoai'], df['Car'])
chi2_car, p_car, dof_car, _ = chi2_contingency(table_car)

# Bảng chéo 2: Phân loại vs Ngày
table_day = pd.crosstab(df['PhanLoai'], df['Day'])
chi2_day, p_day, dof_day, _ = chi2_contingency(table_day)

# In kết quả
print("=== Kiểm định Chi² theo mẫu xe ===")
print(table_car)
print(f"p-value = {p_car:.4f}")
print("=>", "Phụ thuộc vào mẫu xe" if p_car < 0.05 else "Không phụ thuộc vào mẫu xe")

print("\n=== Kiểm định Chi² theo ngày ===")
print(table_day)
print(f"p-value = {p_day:.4f}")
print("=>", "Phụ thuộc vào ngày thử nghiệm" if p_day < 0.05 else "Không phụ thuộc vào ngày thử nghiệm")

=== Kiểm định Chi² theo mẫu xe ===
Car        A  B  C
PhanLoai          
Thap       1  1  1
TrungBinh  3  1  3
Cao        0  2  0
p-value = 0.2730
=> Không phụ thuộc vào mẫu xe

=== Kiểm định Chi² theo ngày ===
Day        1  2  3  4
PhanLoai             
Thap       0  0  1  2
TrungBinh  2  2  2  1
Cao        1  1  0  0
p-value = 0.4126
=> Không phụ thuộc vào ngày thử nghiệm


## Câu 3

In [15]:
# Tạo bảng flights
cursor.execute('''CREATE TABLE IF NOT EXISTS flights (
                flight INT,
                carrier TEXT,
                dep_time INT,
                sched_dep_time INT,
                arr_time INT,
                origin TEXT,
                dest TEXT
                );
''')

In [16]:
# Dữ liệu mẫu câu 3
data = [
    (1545, 'AA', 830, 815, 1045, 'JFK', 'LAX'),   # 8:30 AM -> 10:45 AM
    (1987, 'DL', 1445, 1430, 1730, 'LGA', 'ATL'), # 2:45 PM -> 5:30 PM
    (1023, 'UA', 50, 45, 250, 'EWR', 'ORD'),      # 00:50 AM -> 2:50 AM
    (2312, 'B6', 1123, 1100, 1345, 'JFK', 'MIA'), # 11:23 AM -> 1:45 PM
]

# Chèn dữ liệu vào bảng
cursor.executemany('''
INSERT INTO flights (flight, carrier, dep_time, sched_dep_time, arr_time, origin, dest) VALUES (?, ?, ?, ?, ?, ?, ?);
''', data)

conn.commit()


In [17]:
for row in cursor.execute("SELECT * FROM flights"):
    print(row)

(1545, 'AA', 830, 815, 1045, 'JFK', 'LAX')
(1987, 'DL', 1445, 1430, 1730, 'LGA', 'ATL')
(1023, 'UA', 50, 45, 250, 'EWR', 'ORD')
(2312, 'B6', 1123, 1100, 1345, 'JFK', 'MIA')


In [18]:
cursor.execute('''
SELECT 
    flight,
    carrier,
    origin,
    dest,
    
    printf('%02d:%02d', CAST(dep_time / 100 AS INTEGER), dep_time % 100) AS dep_time,
    printf('%02d:%02d', CAST(sched_dep_time / 100 AS INTEGER), sched_dep_time % 100) AS sched_dep_time,
    printf('%02d:%02d', CAST(arr_time / 100 AS INTEGER), arr_time % 100) AS arr_time

FROM flights;
''')

for row in cursor.fetchall():
    print(f" Flight {row[0]} ({row[1]}): {row[2]} ➡ {row[3]}")
    print(f"  - Actual Departure   : {row[4]}")
    print(f"  - Scheduled Departure: {row[5]}")
    print(f"  - Arrival Time       : {row[6]}\n")


 Flight 1545 (AA): JFK ➡ LAX
  - Actual Departure   : 08:30
  - Scheduled Departure: 08:15
  - Arrival Time       : 10:45

 Flight 1987 (DL): LGA ➡ ATL
  - Actual Departure   : 14:45
  - Scheduled Departure: 14:30
  - Arrival Time       : 17:30

 Flight 1023 (UA): EWR ➡ ORD
  - Actual Departure   : 00:50
  - Scheduled Departure: 00:45
  - Arrival Time       : 02:50

 Flight 2312 (B6): JFK ➡ MIA
  - Actual Departure   : 11:23
  - Scheduled Departure: 11:00
  - Arrival Time       : 13:45



## Câu 4

In [19]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS cau4 (
    id INT PRIMARY KEY,
    value REAL
)
''')

In [20]:
# Chèn dữ liệu mẫu
sample_data = [
    (1, 1), 
    (2, 19), 
    (3, 11), 
    (4, 120), 
    (5, 15),
    (6, 25), 
    (7, 22), 
    (8, 100),  
    (9, 30), 
    (10, 99)
]

cursor.executemany("INSERT INTO cau4 (id, value) VALUES (?, ?)", sample_data)
conn.commit()

In [21]:
for row in cursor.execute("SELECT * FROM cau4"):
    print(row)

(1, 1.0)
(2, 19.0)
(3, 11.0)
(4, 120.0)
(5, 15.0)
(6, 25.0)
(7, 22.0)
(8, 100.0)
(9, 30.0)
(10, 99.0)


In [22]:
# Tính median
cursor.execute("""
SELECT value FROM cau4 ORDER BY value
""")
vals = [row[0] for row in cursor.fetchall()]
n = len(vals)
median = (vals[n // 2] if n % 2 == 1 else (vals[n // 2 - 1] + vals[n // 2]) / 2)

# Tính |xi - median|
abs_devs = [abs(v - median) for v in vals]
abs_devs.sort()
mad = (abs_devs[n // 2] if n % 2 == 1 else (abs_devs[n // 2 - 1] + abs_devs[n // 2]) / 2)

# Chọn các giá trị là ngoại lệ (vượt 1.5 × MAD)
threshold = 1.5 * mad
outliers = [v for v in vals if abs(v - median) > threshold]

# Xuất kết quả
print(f"Median = {median}")
print(f"MAD = {mad}")
print(f"Ngưỡng ngoại lệ (1.5×MAD): ±{threshold}")
print("Giá trị ngoại lệ:", outliers)

Median = 23.5
MAD = 10.5
Ngưỡng ngoại lệ (1.5×MAD): ±15.75
Giá trị ngoại lệ: [1.0, 99.0, 100.0, 120.0]


> Giá trị thứ 1, 4, 8, 10 vượt ngưỡng, là những giá trị ngoại lệ

## Câu 5

In [25]:
# Tạo bảng Patient
cursor.execute("DROP TABLE IF EXISTS Patient")
cursor.execute('''CREATE TABLE Patient (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                last_name TEXT,
                weight REAL,
                height REAL
                );
''')

In [ ]:
# cursor.execute("Delete from Patient")
# conn.commit

<function Connection.commit()>

In [26]:
# Thêm dữ liệu mẫu
data = [
    ('Nghiem', 60, 175),
    ('Nghiem', 60, 176),   # có thể là người trên
    ('Tuan', 69, 185),
    ('Tuan', 72, 185),
    ('Vu', 85, 190),
    ('Vu', 85, 190)        # trùng hoàn toàn
]

cursor.executemany("INSERT INTO Patient (last_name, weight, height) VALUES (?, ?, ?)", data)
conn.commit()

In [27]:
for row in cursor.execute("SELECT * FROM Patient"):
    print(row)

(1, 'Nghiem', 60.0, 175.0)
(2, 'Nghiem', 60.0, 176.0)
(3, 'Tuan', 69.0, 185.0)
(4, 'Tuan', 72.0, 185.0)
(5, 'Vu', 85.0, 190.0)
(6, 'Vu', 85.0, 190.0)


In [31]:
# Truy vấn so sánh Boolean
print("\nKết quả so sánh:")
cursor.execute("""
SELECT 
    p1.id AS id1,
    p2.id AS id2,
    CASE WHEN p1.last_name = p2.last_name THEN 'YES' ELSE 'NO' END AS same_last_name,
    CASE WHEN p1.weight = p2.weight THEN 'YES' ELSE 'NO' END AS same_weight,
    CASE 
        WHEN p1.last_name = p2.last_name AND p1.weight = p2.weight THEN 'Có thể là cùng người'
        ELSE 'Khác người'
    END AS result
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
""")

# In kết quả rõ ràng
for row in cursor.fetchall():
    print(f"ID {row[0]} & ID {row[1]} | Last name: {row[2]} | Weight: {row[3]} | Result: {row[4]}")


Kết quả so sánh:
ID 1 & ID 2 | Last name: YES | Weight: YES | Result: Có thể là cùng người
ID 1 & ID 3 | Last name: NO | Weight: NO | Result: Khác người
ID 1 & ID 4 | Last name: NO | Weight: NO | Result: Khác người
ID 1 & ID 5 | Last name: NO | Weight: NO | Result: Khác người
ID 1 & ID 6 | Last name: NO | Weight: NO | Result: Khác người
ID 2 & ID 3 | Last name: NO | Weight: NO | Result: Khác người
ID 2 & ID 4 | Last name: NO | Weight: NO | Result: Khác người
ID 2 & ID 5 | Last name: NO | Weight: NO | Result: Khác người
ID 2 & ID 6 | Last name: NO | Weight: NO | Result: Khác người
ID 3 & ID 4 | Last name: YES | Weight: NO | Result: Khác người
ID 3 & ID 5 | Last name: NO | Weight: NO | Result: Khác người
ID 3 & ID 6 | Last name: NO | Weight: NO | Result: Khác người
ID 4 & ID 5 | Last name: NO | Weight: NO | Result: Khác người
ID 4 & ID 6 | Last name: NO | Weight: NO | Result: Khác người
ID 5 & ID 6 | Last name: YES | Weight: YES | Result: Có thể là cùng người


> Vậy nếu so sánh dựa trên 2 yếu tố *"last_name"* và *"weight"* thì ta có:
cặp giữa **1 & 2**,  **5 & 6**  có thể là cùng 1 người